In [ ]:
from pulp import *
import pandas as pd

# %load_ext nb_black

# 假设参数
annual_production_target = 100000000  # 年产量目标:10万吨

# 从 Excel 文件中读取数据
cost = pd.read_excel(
    "F:/2 Paper/2 Ammonia/0 press/SI/1 LCOA/data.xlsx", sheet_name="Cost"
)

lifetime = 25
r = 0.075

a = -1
for i in range(lifetime + 1):
    a = a + 1 / ((1 + r) ** i)


# 获取各项成本
solar_capacity_cost = cost.iloc[2, 13]  # CNY/kw
wind_capacity_cost = cost.iloc[5, 13]  # CNY/kw
battery_capacity_cost = cost.iloc[11, 13]  # CNY/kw

electrolyzer_capacity_cost = cost.iloc[8, 13]   # CNY/kg

PV_data = pd.read_excel("F:/2 Paper/2 Ammonia/0 press/SI/1 LCOA/data.xlsx", sheet_name="PV")
nitrogen_capacity_cost = (PV_data.iloc[5, 2]/100000000)*8760
ammonia_capacity_cost = PV_data.iloc[6, 2]/100000000*8760

AWE_water = PV_data.iloc[121, 2] * PV_data.iloc[139, 2]
AWE_catalyst = PV_data.iloc[122, 2] * PV_data.iloc[140, 2]
AWE_chemical_feedstock = PV_data.iloc[123, 2] * PV_data.iloc[141, 2]
AWE_variable_cost = AWE_water + AWE_catalyst + AWE_chemical_feedstock

objective_df = pd.DataFrame()

locations = ["Anhui", "Beijing",'Chongqing','Fujian','Gansu','Guangdong','Guangxi','Guizhou','Hainan','Hebei','Heilongjiang','Henan','Hubei','Hunan','Inner Mongolia','Jiangsu','Jiangxi','Jilin','Liaoning','Ningxia','Qinghai','Shaanxi','Shandong','Shanghai','Shanxi','Sichuan','Tianjin','Xinjiang','Yunnan','Zhejiang']

for i, location in enumerate(locations):

    # 从Excel文件中读取太阳能和风能的容量系数
    weather_profile = pd.read_excel(
        "F:/2 Paper/2 Ammonia/2 Python/4 code/3 LCC/Inputs/weather_profile15.xlsx",
        sheet_name=None,
    )

    # 获取太阳能和风能的容量系数
    solar_capacity_factor = (weather_profile["solar"].iloc[0:8760, i + 1]).values.tolist()
    wind_capacity_factor = (weather_profile["wind"].iloc[0:8760, i + 1]).values.tolist()

    # 创建线性规划问题
    prob = LpProblem("GreenAmmonia", LpMinimize)

    # 定义决策变量
    solar_capacity = LpVariable("Solar_Capacity", lowBound=0)
    wind_capacity = LpVariable("Wind_Capacity", lowBound=0)
    battery_capacity = LpVariable("Battery_Capacity", lowBound=0)
    electrolyzer_capacity = LpVariable("Electrolyzer_Capacity", lowBound=0)
    nitrogen_capacity = LpVariable("Nitrogen_Capacity", lowBound=0)
    ammonia_capacity = LpVariable("Ammonia_Capacity", lowBound=0)
    CAPEX = LpVariable("CAPEX", lowBound=0)
    CAPEX_solar = LpVariable("CAPEX_solar", lowBound=0)
    CAPEX_wind = LpVariable("CAPEX_wind", lowBound=0)
    CAPEX_battery = LpVariable("CAPEX_battery", lowBound=0)
    CAPEX_electrolyzer = LpVariable("CAPEX_electrolyzer", lowBound=0)
    CAPEX_nitrogen = LpVariable("CAPEX_nitrogen", lowBound=0)
    CAPEX_ammonia = LpVariable("CAPEX_ammonia", lowBound=0)
    CAPEX_all = LpVariable("CAPEX_all", lowBound=0)

    # 每小时产出量决策变量
    solar_production = LpVariable.dicts(
        "Solar_Production", range(1, 8760 + 1), lowBound=0
    )
    wind_production = LpVariable.dicts(
        "Wind_Production", range(1, 8760 + 1), lowBound=0
    )
    battery_discharge = LpVariable.dicts(
        "Battery_Discharge", range(1, 8760 + 1), lowBound=0
    )
    battery_charge = LpVariable.dicts("Battery_Charge", range(1, 8760 + 1), lowBound=0)
    electricity_curtail = LpVariable.dicts(
        "Electricity_Curtail", range(1, 8760 + 1), lowBound=0
    )
    electricity_consumption = LpVariable.dicts(
        "Electricity_Consumption", range(1, 8760 + 1), lowBound=0
    )
    battery_soc = LpVariable.dicts("Battery_Soc", range(1, 8760 + 1), lowBound=0)

    hydrogen_production = LpVariable.dicts(
        "Hydrogen_Production", range(1, 8760 + 1), lowBound=0
    )

    nitrogen_production = LpVariable.dicts(
        "Nitrogen_Production", range(1, 8760 + 1), lowBound=0
    )
    ammonia_production = LpVariable.dicts(
        "Ammonia_Production", range(1, 8760 + 1), lowBound=0
    )

    # 定义目标函数

    prob += CAPEX_solar == solar_capacity * solar_capacity_cost
    prob += CAPEX_wind == wind_capacity * wind_capacity_cost
    prob += CAPEX_battery == battery_capacity * battery_capacity_cost
    prob += CAPEX_electrolyzer == electrolyzer_capacity * electrolyzer_capacity_cost
    prob += CAPEX_nitrogen == nitrogen_capacity * nitrogen_capacity_cost
    prob += CAPEX_ammonia == ammonia_capacity * ammonia_capacity_cost

    prob += (
        CAPEX
        == CAPEX_solar
        + CAPEX_wind
        + CAPEX_battery
        + CAPEX_electrolyzer
        + CAPEX_nitrogen
        + CAPEX_ammonia
    )

    prob += (
        CAPEX
        + (
            CAPEX_solar * 0.03
            + CAPEX_wind * 0.02
            + CAPEX_battery * 0.025
            + CAPEX_electrolyzer * 0.025
            + ammonia_capacity * ammonia_capacity_cost * 0.025
            + nitrogen_capacity * nitrogen_capacity_cost * 0.025
            + AWE_variable_cost * annual_production_target 
        )
        * a
        - (CAPEX * 0.05) / (1 + r) ** lifetime
    ) / (annual_production_target * a)
    
    prob += CAPEX_all ==(CAPEX- (CAPEX * 0.05) / (1 + r) ** lifetime ) / (annual_production_target * a)

    # 年氨产量约束
    prob += lpSum(ammonia_production.values()) == annual_production_target

    # 光伏板产量约束
    for t in range(1, 8760 + 1):
        prob += solar_production[t] == solar_capacity * solar_capacity_factor[t - 1]
    # 风电涡轮机产量约束
    for t in range(1, 8760 + 1):
        prob += wind_production[t] == wind_capacity * wind_capacity_factor[t - 1]

    # 氮气产量约束
    for t in range(1, 8760 + 1):
        prob += nitrogen_production[t] <= nitrogen_capacity

    # 合成氨产量约束
    for t in range(1, 8760 + 1):
        prob += ammonia_production[t] <= ammonia_capacity
        
    # 氢气产量约束
    for t in range(1, 8760 + 1):
        prob += hydrogen_production[t] <= electrolyzer_capacity
    

    # 电力放电量和充电量的约束

    for t in range(1, 8760 + 1):
        prob += battery_discharge[t] <= battery_capacity  # 限制放电量不超过电池容量
        prob += battery_charge[t] <= battery_capacity  # 限制充电量不超过电池容量
        prob += battery_soc[t] <= battery_capacity  # 剩余电量不超过电池容量
    for t in range(2, 8760 + 1):
        prob += battery_soc[t] == battery_charge[t] - battery_discharge[t] + battery_soc[t - 1] 

        prob += (
            battery_soc[1]
            == (
                solar_production[1]
                + wind_production[1]
                - electricity_consumption[1]
                - electricity_curtail[1]
            )
        )
        prob +=  battery_charge[1] == battery_soc[1]
        prob += battery_discharge[1] == 0

    # 电力消耗约束
    for t in range(1, 8760 + 1):

        prob += electricity_consumption[t] == (
            solar_production[t]
            + wind_production[t]
            + battery_discharge[t]
            - battery_charge[t]
            - electricity_curtail[t]
        )
    for t in range(1, 8760 + 1):
        prob += electricity_consumption[t] == (
            51.8 * hydrogen_production[t]
            + 0.11 * nitrogen_production[t]
            + 0.47473 * ammonia_production[t]
        )

    # 单位绿氨需要投入多少氢气
    for t in range(1, 8760 + 1):
        prob += hydrogen_production[t] >= 0.18878 * ammonia_production[t]
    # 单位绿氨需要投入多少氮气
    for t in range(1, 8760 + 1):
        prob += ammonia_production[t] * 0.87445 <= nitrogen_production[t]

    # 解决线性规划问题
    prob.solve()

    # 打印结果
    print("Status:", LpStatus[prob.status])

    print("Total Cost =", value(prob.objective))

    # 创建一个空的 DataFrame
    output_df = pd.DataFrame()

    # 将每小时的产量添加到 DataFrame 中
    output_df["Hour"] = range(1, 8760 + 1)
    output_df["Solar_Production"] = [
        solar_production[t].varValue for t in range(1, 8760 + 1)
    ]
    output_df["Wind_Production"] = [
        wind_production[t].varValue for t in range(1, 8760 + 1)
    ]
    output_df["Battery_Charge"] = [
        battery_charge[t].varValue for t in range(1, 8760 + 1)
    ]
    output_df["Battery_Discharge"] = [
        battery_discharge[t].varValue for t in range(1, 8760 + 1)
    ]
    output_df["Battery_Soc"] = [battery_soc[t].varValue for t in range(1, 8760 + 1)]
    output_df["Electricity_Curtail"] = [electricity_curtail[t].varValue for t in range(1, 8760 + 1)]
    output_df["Hydrogen_Production"] = [
        hydrogen_production[t].varValue for t in range(1, 8760 + 1)
    ]
    output_df["Nitrogen_Production"] = [
        nitrogen_production[t].varValue for t in range(1, 8760 + 1)
    ]
    output_df["Ammonia_Production"] = [
        ammonia_production[t].varValue for t in range(1, 8760 + 1)
    ]

    # 将 DataFrame 写入 Excel 文件
    output_file_path = f"F:/2 Paper/2 Ammonia/2 Python/4 code/3 LCC/Inputs/resultsv20/2060/capacity_factor_正12%/{location}_output.xlsx"
    output_df.to_excel(output_file_path, index=False)
    print("DataFrame 已保存至:", output_file_path)

    # 创建一个空的 DataFrame
    capacity_df = pd.DataFrame()

    # 添加每种技术的容量到 DataFrame 中
    capacity_df["Solar_Capacity"] = [solar_capacity.varValue]
    capacity_df["Wind_Capacity"] = [wind_capacity.varValue]
    capacity_df["Battery_Capacity"] = [battery_capacity.varValue]
    capacity_df["Electrolyzer_Capacity"] = [electrolyzer_capacity.varValue]
    capacity_df["Nitrogen_Capacity"] = [nitrogen_capacity.varValue]
    capacity_df["Ammonia_Capacity"] = [ammonia_capacity.varValue]
    capacity_df["CAPEX_all"] = [CAPEX_all.varValue]

    # 显示结果
    print(capacity_df)

    # 输出到 Excel 文件
    output_file_path = f"F:/2 Paper/2 Ammonia/2 Python/4 code/3 LCC/Inputs/resultsv20/2060/capacity_factor_正12%/{location}_output_capacity.xlsx"
    capacity_df.to_excel(output_file_path, index=False)
    print("DataFrame 已保存到文件:", output_file_path)

    # Store the objective function value in the DataFrame using pd.concat
    new_data = pd.DataFrame({"Location": [location], "2060": [value(prob.objective)]})
    objective_df = pd.concat([objective_df, new_data], ignore_index=True)

objective_df.to_excel("F:/2 Paper/2 Ammonia/2 Python/4 code/3 LCC/Inputs/resultsv20/2060/capacity_factor_正12%/0 objective_cost_2060.xlsx", index=False)